[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/TotoroUI-jupyter/blob/main/test.ipynb)

In [ ]:
!pip install -q torchsde

%cd /content
!git clone -b totoro https://github.com/camenduru/ComfyUI /content/TotoroUI
!git clone -b totoro https://github.com/camenduru/ComfyUI_IPAdapter_plus /content/TotoroUI/IPAdapter

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/128713 -d /content/TotoroUI/models -o dreamshaper_8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/TotoroUI/models/clip_vision -o CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter-plus_sd15.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/8744d2de-e74b-4b68-b46d-e3a8bd649cdf -d /content -o ip1.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/e42ff968-2298-4a65-b685-1e9c0c0eb2cf -d /content -o ip2.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/c4ed3b91-5cd6-4465-abd9-6f90c34ea61b -d /content -o color_mask.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/dae2c8b3-3ddd-4162-84b2-19f5b9c6f5fd -d /content -o back.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Ttl/ComfyUi_NNLatentUpscale/raw/master/sd15_resizer.pt -d /content/TotoroUI/models -o sd15_resizer.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Ttl/ComfyUi_NNLatentUpscale/raw/master/sdxl_resizer.pt -d /content/TotoroUI/models -o sdxl_resizer.pt

%cd /content/TotoroUI
import torch
import numpy as np
from PIL import Image
import totoro
import nodes
import sys
sys.path.append('/content/TotoroUI/IPAdapter')
import IPAdapterPlus
import scipy
from latent_resizer import LatentResizer
from totoro import model_management

with torch.no_grad():
  model_patcher, clip, vae, clipvision = totoro.sd.load_checkpoint_guess_config("/content/TotoroUI/models/dreamshaper_8.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

# mask_from_colors() and conditioning_combine_multiple() from https://github.com/cubiq/ComfyUI_essentials/blob/main/essentials.py
def mask_from_colors(image, threshold_r, threshold_g, threshold_b, remove_isolated_pixels, fill_holes):
    red = ((image[..., 0] >= 1-threshold_r) & (image[..., 1] < threshold_g) & (image[..., 2] < threshold_b)).float()
    green = ((image[..., 0] < threshold_r) & (image[..., 1] >= 1-threshold_g) & (image[..., 2] < threshold_b)).float()
    blue = ((image[..., 0] < threshold_r) & (image[..., 1] < threshold_g) & (image[..., 2] >= 1-threshold_b)).float()
    cyan = ((image[..., 0] < threshold_r) & (image[..., 1] >= 1-threshold_g) & (image[..., 2] >= 1-threshold_b)).float()
    magenta = ((image[..., 0] >= 1-threshold_r) & (image[..., 1] < threshold_g) & (image[..., 2] > 1-threshold_b)).float()
    yellow = ((image[..., 0] >= 1-threshold_r) & (image[..., 1] >= 1-threshold_g) & (image[..., 2] < threshold_b)).float()
    black = ((image[..., 0] <= threshold_r) & (image[..., 1] <= threshold_g) & (image[..., 2] <= threshold_b)).float()
    white = ((image[..., 0] >= 1-threshold_r) & (image[..., 1] >= 1-threshold_g) & (image[..., 2] >= 1-threshold_b)).float()
    if remove_isolated_pixels > 0 or fill_holes:
        colors = [red, green, blue, cyan, magenta, yellow, black, white]
        color_names = ['red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'black', 'white']
        processed_colors = {}
        for color_name, color in zip(color_names, colors):
            color = color.cpu().numpy()
            masks = []

            for i in range(image.shape[0]):
                mask = color[i]
                if remove_isolated_pixels > 0:
                    mask = scipy.ndimage.binary_opening(mask, structure=np.ones((remove_isolated_pixels, remove_isolated_pixels)))
                if fill_holes:
                    mask = scipy.ndimage.binary_fill_holes(mask)
                mask = torch.from_numpy(mask)
                masks.append(mask)
            processed_colors[color_name] = torch.stack(masks, dim=0).float()
        red = processed_colors['red']
        green = processed_colors['green']
        blue = processed_colors['blue']
        cyan = processed_colors['cyan']
        magenta = processed_colors['magenta']
        yellow = processed_colors['yellow']
        black = processed_colors['black']
        white = processed_colors['white']
        del colors, processed_colors
    return (red, green, blue, cyan, magenta, yellow, black, white,)

def conditioning_combine_multiple(conditioning_1, conditioning_2, conditioning_3=None, conditioning_4=None, conditioning_5=None):
    c = conditioning_1 + conditioning_2
    if conditioning_3 is not None:
        c += conditioning_3
    if conditioning_4 is not None:
        c += conditioning_4
    if conditioning_5 is not None:
        c += conditioning_5
    return (c,)

def upscale(latent, upscale):
    device = model_management.get_torch_device()
    samples = latent.to(device=device, dtype=torch.float16)
    model = LatentResizer.load_model('/content/TotoroUI/models/sd15_resizer.pt', device, torch.float16)
    model.to(device=device)
    latent_out = (model(0.13025 * samples, scale=upscale) / 0.13025)
    latent_out = latent_out.to(device="cpu")
    model.to(device=model_management.vae_offload_device())
    return ({"samples": latent_out},)

IPAdapterPlus_model = IPAdapterPlus.IPAdapterUnifiedLoader().load_models(model_patcher, 'PLUS (high strength)', lora_strength=0.0, provider="CPU", ipadapter=None)

In [ ]:
color_image, color_mask = nodes.LoadImage().load_image("/content/color_mask.png")

output1_image, output1_mask = nodes.LoadImage().load_image("/content/ip1.png")
tokens_1 = clip.tokenize("illustration of a blond woman")
cond_1, pooled_1 = clip.encode_from_tokens(tokens_1, return_pooled=True)
cond_1 = [[cond_1, {"pooled_output": pooled_1}]]
n_tokens_1 = clip.tokenize("anime")
n_cond_1, n_pooled_1 = clip.encode_from_tokens(n_tokens_1, return_pooled=True)
n_cond_1 = [[n_cond_1, {"pooled_output": n_pooled_1}]]

output2_image, output2_mask = nodes.LoadImage().load_image("/content/ip2.png")
tokens_2 = clip.tokenize("anime illustration of a young woman with a black jacket")
cond_2, pooled_2 = clip.encode_from_tokens(tokens_2, return_pooled=True)
cond_2 = [[cond_2, {"pooled_output": pooled_2}]]
n_tokens_2 = clip.tokenize("")
n_cond_2, n_pooled_2 = clip.encode_from_tokens(n_tokens_2, return_pooled=True)
n_cond_2 = [[n_cond_2, {"pooled_output": n_pooled_2}]]

output3_image, output3_mask = nodes.LoadImage().load_image("/content/back.png")
tokens_3 = clip.tokenize("closeup of two girl friends shopping in a sci-fi space station")
cond_3, pooled_3 = clip.encode_from_tokens(tokens_3, return_pooled=True)
cond_3 = [[cond_3, {"pooled_output": pooled_3}]]
n_tokens_3 = clip.tokenize("blurry, lowres, bad art, ill, distorted, malformed, horror")
n_cond_3, n_pooled_3 = clip.encode_from_tokens(n_tokens_3, return_pooled=True)
n_cond_3 = [[n_cond_3, {"pooled_output": n_pooled_3}]]

red, green, blue, cyan, magenta, yellow, black, white = mask_from_colors(image=color_image, threshold_r=0.15, threshold_g=0.15, threshold_b=0.15, remove_isolated_pixels=0, fill_holes=False)
params_1, positive_1, negative_1 = IPAdapterPlus.IPAdapterRegionalConditioning().conditioning(output1_image, image_weight=0.7, prompt_weight=1.0, weight_type='linear', start_at=0.0, end_at=1.0, mask=red, positive=cond_1, negative=n_cond_1)
params_2, positive_2, negative_2 = IPAdapterPlus.IPAdapterRegionalConditioning().conditioning(output2_image, image_weight=0.7, prompt_weight=1.0, weight_type='linear', start_at=0.0, end_at=1.0, mask=green, positive=cond_2, negative=n_cond_2)
params_3, positive_3, negative_3 = IPAdapterPlus.IPAdapterRegionalConditioning().conditioning(output3_image, image_weight=0.7, prompt_weight=1.0, weight_type='linear', start_at=0.0, end_at=1.0, mask=black, positive=None, negative=None)

positive = conditioning_combine_multiple(conditioning_1=positive_1, conditioning_2=positive_2, conditioning_3=cond_3)
negative = conditioning_combine_multiple(conditioning_1=negative_1, conditioning_2=negative_2, conditioning_3=n_cond_3)

ipadapter_params = IPAdapterPlus.IPAdapterCombineParams().combine(params_1=params_1, params_2=params_2, params_3=params_3)

ip_model_patcher = IPAdapterPlus.IPAdapterAdvanced().apply_ipadapter(IPAdapterPlus_model[0], IPAdapterPlus_model[1], start_at=0.0, end_at=1.0, weight=1.0, weight_style=1.0, weight_composition=1.0, expand_style=False, weight_type="linear", combine_embeds="concat", embeds_scaling='V only', ipadapter_params=ipadapter_params[0])

latent = {"samples":torch.zeros([1, 4, 512 // 8, 768 // 8])}
seed = 3
sample = nodes.common_ksampler(model=ip_model_patcher[0], 
                          seed=seed, 
                          steps=30, 
                          cfg=7.0, 
                          sampler_name="dpmpp_2m", 
                          scheduler="karras", 
                          positive=positive[0], 
                          negative=negative[0],
                          latent=latent, 
                          denoise=1)

latent_up = upscale(sample[0]["samples"].to(torch.float16), 1.5)

sample_up = nodes.common_ksampler(model=ip_model_patcher[0], 
                          seed=seed, 
                          steps=30, 
                          cfg=7.0, 
                          sampler_name="dpmpp_2m", 
                          scheduler="karras", 
                          positive=positive[0], 
                          negative=negative[0],
                          latent=latent_up[0], 
                          denoise=0.55)

with torch.inference_mode():
  sample_up = sample_up[0]["samples"].to(torch.float16)
  vae.first_stage_model.cuda()
  decoded = vae.decode_tiled(sample_up).detach()

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])